# Даты: основные типы

## → Любой анализ событий во времени подразумевает работу с датой или временем.

## Для них в Postgres существует несколько типов данных — все они представлены в таблице ниже.

|Имя	|Размер	|Описание	|Наименьшее значение	|Наибольшее значение|	Точность|
|-|-|-|-|-|-|
|timestamp  [ without time zone ]|	8 байт|	дата и время (без часового пояса)|	4713 до н. э.|	294276 н. э.|	1 микросекунда / 14 цифр|
|timestamp with time zone	|8 байт|	дата и время (с часовым поясом)|	4713 до н. э.|	294276 н. э.|	1 микросекунда / 14 цифр|
|date|	4 байта	|дата (без времени суток)	|4713 до н. э.	|5874897 н. э.|	1 день|
|time [ without time zone ]	|8 байт|	время суток (без даты)|	0:00:00	|24:00:00	|1 микросекунда / 14 цифр|
|time with time zone	|12 байт	|только время суток (с часовым поясом)|	00:00:00+1459	|24:00:00-1459	|1 микросекунда / 14 цифр|
|interval 	|16 байт	|временной интервал	|-178000000 лет	|178000000 лет	|1 микросекунда / 14 цифр|

## TIMESTAMP

### Timestamp — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

### Согласно стандарту ISO, значение выглядит как "2019-07-14 01:35:44.702165+00", где перечислены через точку год-месяц-день, время и часовой пояс.

### Для получения текущего значения даты и времени в Postgres используются функции CURRENT_TIMESTAMP (есть в стандарте SQL) и NOW() (есть в большинстве баз данных).

### Выполните запросы SELECT NOW() и SELECT CURRENT_TIMESTAMP.

### Глядя на вывод, вы заметите, что время отличается на целое число часов от реального времени у вас (если вы не в Лондоне или, скажем, на Канарских островах). В Москве разница составит 3 часа.

### Почему так?

### Давайте разберёмся.

## TIMESTAMP WITH TIME ZONE

### Timestamp with time zone позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

### Предположим, в вашей компании в базу подтягивается время прихода сотрудников на работу. Вы пришли в 10 утра по Москве, а в Екатеринбурге в это время — полдень. Чтобы ваши коллеги из Екатеринбурга поняли, что вы пришли на работу вовремя, им нужно помнить про разницу в часовых поясах. А теперь представим, что пользователи БД разбросаны по всему миру и всем им необходимо помнить о разнице во времени и учитывать её при сверках с другими регионами.

### Согласитесь, неудобно. Поэтому временные метки лучше всего хранить в базе в едином часовом поясе.

### Посмотрим, как работает этот тип данных.

### Сначала попробуем узнать, в каком часовом поясе выводятся временные данные в настоящий момент. Для этого выполните команду

* ### show timezone
### В результате вы увидите GMT — это наиболее частая установка по умолчанию для баз данных. 

### Посмотрите содержимое этого справочника в Metabase.

### А ещё посмотрите ваше время в каком-нибудь часовом поясе, например, в Москве. Для этого выполните в Metabase запрос

* ### select now() at time zone 'Europe/Moscow'
### Указание at time zone позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.

### В таблице ниже приведены примеры того, как работает at time zone для разных типов данных.

## DATE

### С типом date вы уже знакомы, его реализация предельно проста. Отметим только, что тип timestamp (with/without time zone) можно легко перевести в соответствующую дату, используя синтаксис

* ### "timestamp_column"::date
### И наоборот, тип date преобразуется в timestamp (дата и 00:00:00) с помощью

* ### "date_column"::timestamp
### Для получения текущей даты можно использовать

* ### select CURRENT_DATE
### или

* ### select now()::date

## INTERVAL

### Interval — тип данных, позволяющий хранить разницу между двумя временными метками. 

### Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за того, что количество дней в месяце и часов в дне может быть разным. Пример значения такого типа: "195 days -10:52:23.563955".

### Подробнее о том, как работают интервалы, мы узнаем позже, на примере конкретных функций, с помощью которых можно вычислять эти интервалы и работать с ними.